In [ ]:
import numpy as np 
import pandas as pd 

### Version 1: dowload full excel

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
# Function to download XLS files from a given URL
def download_xls_files_from_page(url):
    response = requests.get(url)
    response.raise_for_status()  # Check that the request was successful

    soup = BeautifulSoup(response.text, 'html.parser')
    xls_links = soup.find_all('a', href=lambda x: (x.endswith('.xls') if x else False))

    for link in xls_links:
        file_url = link['href']
        file_name = file_url.split('/')[-1]  # Extract the file name

        with requests.get(file_url, stream=True) as file_response:
            file_response.raise_for_status()
            with open(file_name, 'wb') as f:
                for chunk in file_response.iter_content(chunk_size=8192): 
                    f.write(chunk)
        print(f"Downloaded {file_name}")


In [ ]:
# Base URL and page template
base_url = 'https://usda.library.cornell.edu/concern/publications/3t945q76s'
page_param = '?locale=en&page='

# Determine the number of pages or set a high number if unknown
# You can manually set this, or dynamically find it if the number of pages is listed on the site
number_of_pages = 3  # Example number, you should set this appropriately

# Loop through all the pages
for page in range(1, number_of_pages + 1):
    page_url = f"{base_url}{page_param}{page}#release-items"
    print(f"Scraping {page_url}")
    download_xls_files_from_page(page_url)

### version 2: only download corn snd

In [ ]:
import requests
from bs4 import BeautifulSoup
import xlrd
import openpyxl
from tempfile import NamedTemporaryFile

In [ ]:
def download_and_extract_cells(file_url, sheet_name, cell_range):
    response = requests.get(file_url)
    response.raise_for_status()

    # Write the file temporarily to disk
    with NamedTemporaryFile(delete=False, suffix='.xls') as tmp:
        temp_file_name = tmp.name
        tmp.write(response.content)
    
    # Read the specific range from the .xls file
    book = xlrd.open_workbook(temp_file_name)
    sheet = book.sheet_by_name(sheet_name)

    data = []
    for row_idx in range(cell_range['start_row'] - 1, cell_range['end_row']):
        row_data = []
        for col_idx in range(cell_range['start_col'] - 1, cell_range['end_col']):
            row_data.append(sheet.cell_value(row_idx, col_idx))
        data.append(row_data)
    
    return data

def save_to_excel(data, output_filename):
    # Create a new Workbook
    wb = openpyxl.Workbook()
    ws = wb.active

    # Write data to Workbook
    for row_idx, row_data in enumerate(data, start=1):
        for col_idx, value in enumerate(row_data, start=1):
            ws.cell(row=row_idx, column=col_idx, value=value)
    
    # Save the workbook
    wb.save(output_filename)

In [ ]:
# Specify the range and sheet name
cell_range = {'start_row': 30, 'end_row': 50, 'start_col': 1, 'end_col': 5}  # A30:E50
sheet_name = 'Page 12'

# Specify the base URL and parameters
base_url = 'https://usda.library.cornell.edu/concern/publications/3t945q76s'
page_param = '?locale=en&page='

# Specify the number of pages
number_of_pages = 2  # Adjust as needed

for page in range(1, number_of_pages + 1):
    page_url = f"{base_url}{page_param}{page}#release-items"
    soup = BeautifulSoup(requests.get(page_url).text, 'html.parser')
    xls_links = soup.find_all('a', href=lambda x: (x and x.endswith('.xls')))

    for link in xls_links:
        file_url = link['href']
        data = download_and_extract_cells(file_url, sheet_name, cell_range)

        # Generate a filename for the new .xlsx file
        file_name = file_url.split('/')[-1]
        new_file_name = f"{file_name.split('.')[0]}_reduced.xlsx"
        
        # Save the data to a new .xlsx file
        save_to_excel(data, new_file_name)
        print(f"Saved data to {new_file_name}")
